In [1]:
import torch as t
import numpy as np

# 生成数据集
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = t.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=t.float)
labels = true_w[0]*features[:, 0]+true_w[1]*features[:, 1]+true_b
labels += t.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=t.float)

# 读取数据
import torch.utils.data as Data
batch_size = 10
# 将数据集的特征和标签组合
dataset = Data.TensorDataset(features, labels)
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

for X, y in data_iter:
    print(X, y)
    break
    

tensor([[-0.4236, -0.8784],
        [ 0.9715, -0.5210],
        [ 0.5580,  1.2805],
        [ 2.2667,  1.0517],
        [-0.2790,  0.3837],
        [ 1.6711, -2.2000],
        [-0.1990, -0.0023],
        [-0.1184,  0.7691],
        [ 1.3348, -1.6525],
        [ 0.7780, -0.2597]]) tensor([ 6.3564,  7.9196,  0.9586,  5.1599,  2.3379, 15.0358,  3.8005,  1.3427,
        12.4873,  6.6189])


In [3]:
#定义模型
import torch.nn as nn

class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
        
    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net)

# 事实上可以使用sequential 来搭建网络
# 写法1
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    #此处可以添加层
)

# 写法2
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))

#写法3
from collections import OrderedDict
net = nn.Sequential(OrderedDict(
    [
        ('linear', nn.Linear(num_inputs, 1))
    ]
))

print(net)
print(net[0])


LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [5]:
# 查看可以学习的参数
for param in net.parameters():
    print(param)
    
# 初始化参数
from torch.nn import init
init.normal_(net[0].weight, mean=0, std=0.01)
init.normal_(net[0].bias, val=0)



Parameter containing:
tensor([[ 0.5130, -0.5863]], requires_grad=True)
Parameter containing:
tensor([-0.6543], requires_grad=True)


In [6]:
#定义损失函数
loss = nn.MSELoss()
# 定义优化函数
import torch.optim as optim

optimzer = optim.SGD(net.parameters(), lr=0.03)
print(optimzer)


SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [8]:
#训练模型
num_epochs = 3
for epoch in range(1, num_epochs+1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimzer.zero_grad()
        l.backward()
        optimzer.step()
        
    print("epoch %d, loss : %f" %(epoch, l.item()))
    
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)


epoch 1, loss : 0.000184
epoch 2, loss : 0.000031
epoch 3, loss : 0.000047
[2, -3.4] Parameter containing:
tensor([[ 2.0002, -3.3995]], requires_grad=True)
4.2 Parameter containing:
tensor([4.1997], requires_grad=True)
